In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd '/content/drive/My Drive/Deep active'
! ls

/content/drive/My Drive/Deep active
cifar10_2.ipynb  cifar10.ipynb		  model.h5
cifar10_3.ipynb  model_architecture.json  Untitled0.ipynb


In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.python.keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

Using TensorFlow backend.


In [0]:
deep_active_pretrain = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [0]:
deep_active_pretrain.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_valid=x_train[40000:]
y_valid=y_train[40000:]
x_train=x_train[:40000]
y_train=y_train[:40000]
x_train = x_train / 255.0
x_test = x_test / 255.0
x_valid=x_valid/255.0

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
y_valid = np_utils.to_categorical(y_valid, 10)
print(x_train.shape)
print(x_test.shape)
print(x_valid.shape)

170500096/170498071 [==============================] - 3s 0us/step
(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
from keras.callbacks import ModelCheckpoint
import os
model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(deep_active_pretrain)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])
file_path="model-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath=file_path, verbose=1 )
callbacks_list = [checkpoint]


In [0]:
model.summary()

ValueError: ignored

In [0]:
history = model.fit(x_train, y_train, epochs=3, batch_size=32, validation_data=(x_valid, y_valid), callbacks=callbacks_list)


Train on 40000 samples, validate on 10000 samples
Epoch 1/3
39968/40000 [============================>.] - ETA: 0s - loss: 0.2084 - acc: 0.9265
Epoch 00001: saving model to model-01-0.97.h5
40000/40000 [==============================] - 1345s 34ms/sample - loss: 0.2084 - acc: 0.9265 - val_loss: 0.0962 - val_acc: 0.9678
Epoch 2/3
39968/40000 [============================>.] - ETA: 0s - loss: 0.1339 - acc: 0.9534
Epoch 00002: saving model to model-02-0.98.h5
40000/40000 [==============================] - 1315s 33ms/sample - loss: 0.1339 - acc: 0.9534 - val_loss: 0.0661 - val_acc: 0.9791
Epoch 3/3
39968/40000 [============================>.] - ETA: 0s - loss: 0.1015 - acc: 0.9666
Epoch 00003: saving model to model-03-0.98.h5
40000/40000 [==============================] - 1316s 33ms/sample - loss: 0.1014 - acc: 0.9667 - val_loss: 0.0529 - val_acc: 0.9837


In [0]:
print(model.evaluate(x_test,y_test))


10000/10000 [==============================] - 88s 9ms/sample - loss: 0.0556 - acc: 0.9830
[0.05555110847055912, 0.98297]
